In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
from glob import glob
from tqdm import tqdm
import json
import copy
import os
import torchaudio
import penn

hopsize = .01
fmin = 30.
fmax = 1000.
gpu = 0
batch_size = 2048
checkpoint = None

center = 'half-hop'
interp_unvoiced_at = .065

In [3]:
text = []
files = sorted(glob('osman-news-edge-tts-text/*.json'))
for f in files:
    with open(f) as fopen:
        text.append(json.load(fopen))

filtered = []
for i in tqdm(range(len(text))):
    t = copy.copy(text[i])
    if '---' in t['normalized'] or '~' in t['normalized']:
        continue
    new_filename = os.path.join('generate-husein-news-normalized-v2', f'{i}.mp3')
    if not os.path.exists(new_filename):
        continue
    
    t['audio_filename'] = new_filename
    filtered.append(t)
    
len(filtered)

100%|███████████████████████████████████████████████████████████████████████████████| 49985/49985 [00:00<00:00, 168093.52it/s]


22864

In [4]:
text = []
files = sorted(glob('osman-parliament-edge-tts-text/*.json'))
for f in files:
    with open(f) as fopen:
        text.append(json.load(fopen))
        
for i in tqdm(range(len(text))):
    t = copy.copy(text[i])
    if '---' in t['normalized'] or '~' in t['normalized']:
        continue
    new_filename = os.path.join('generate-husein-wiki-normalized-v2', f'{i}.mp3')
    if not os.path.exists(new_filename):
        continue
    
    t['audio_filename'] = new_filename
    filtered.append(t)
    
len(filtered)

100%|███████████████████████████████████████████████████████████████████████████████| 59582/59582 [00:00<00:00, 237719.74it/s]


40556

In [5]:
with open('chatbot-conversation-politics.json') as fopen:
    text = json.load(fopen)
        
for i in tqdm(range(len(text))):
    t = copy.copy(text[i])
    if '---' in t['normalized'] or '~' in t['normalized']:
        continue
    new_filename = os.path.join('husein-chatbot-politics-normalized-v2', f'{i}.mp3')
    if not os.path.exists(new_filename):
        continue
    
    t['audio_filename'] = new_filename
    filtered.append(t)

100%|█████████████████████████████████████████████████████████████████████████████| 272785/272785 [00:01<00:00, 251169.24it/s]


In [6]:
with open('chatbot-conversation.json') as fopen:
    text = json.load(fopen)
        
for i in tqdm(range(len(text))):
    t = copy.copy(text[i])
    if '---' in t['normalized'] or '~' in t['normalized']:
        continue
    new_filename = os.path.join('husein-chatbot-normalized-v2', f'{i}.mp3')
    if not os.path.exists(new_filename):
        continue
    
    t['audio_filename'] = new_filename
    filtered.append(t)

100%|███████████████████████████████████████████████████████████████████████████████| 58069/58069 [00:00<00:00, 200650.86it/s]


In [7]:
with open('news-politics.json') as fopen:
    text = json.load(fopen)
        
for i in tqdm(range(len(text))):
    t = copy.copy(text[i])
    if '---' in t['normalized'] or '~' in t['normalized']:
        continue
    new_filename = os.path.join('husein-news-politics-normalized-v2', f'{i}.mp3')
    if not os.path.exists(new_filename):
        continue
    
    t['audio_filename'] = new_filename
    filtered.append(t)

100%|█████████████████████████████████████████████████████████████████████████████| 130786/130786 [00:00<00:00, 277809.78it/s]


In [8]:
len(filtered)

132779

In [9]:
import penn

hopsize = .01
fmin = 30.
fmax = 1000.
gpu = 0
batch_size = 2048
checkpoint = None

center = 'half-hop'
interp_unvoiced_at = .065

In [13]:
pitch, periodicity = penn.from_audio(
    *torchaudio.load(filtered[0]['audio_filename']),
    hopsize=hopsize,
    fmin=fmin,
    fmax=fmax,
    checkpoint=checkpoint,
    batch_size=batch_size,
    center=center,
    interp_unvoiced_at=interp_unvoiced_at,
    gpu=gpu)
pitch[0][pitch[0] > 300].shape[0]

0

In [14]:
filtered_filtered = []
for f in tqdm(filtered):
    pitch, periodicity = penn.from_audio(
        *torchaudio.load(f['audio_filename']),
        hopsize=hopsize,
        fmin=fmin,
        fmax=fmax,
        checkpoint=checkpoint,
        batch_size=batch_size,
        center=center,
        interp_unvoiced_at=interp_unvoiced_at,
        gpu=gpu)
    if pitch[0][pitch[0] > 300].shape[0] > 1:
        continue
    
    filtered_filtered.append(f)

100%|█████████████████████████████████████████████████████████████████████████████████| 132779/132779 [57:39<00:00, 38.38it/s]


In [15]:
len(filtered_filtered), len(filtered)

(127154, 132779)

In [16]:
import soundfile as sf

def get_length(file):
    y, sr = sf.read(file)
    return len(y) / sr

In [17]:
import mp

def loop(files):
    files, _ = files
    length = 0
    for f in tqdm(files):
        try:
            length += get_length(f['audio_filename'])
        except:
            pass
    return [length]

In [18]:
lengths = mp.multiprocessing(filtered_filtered, loop, cores = 10)

100%|██████████████████████████████████████████████████████████████████████████████████| 12715/12715 [00:42<00:00, 296.77it/s]


In [19]:
import numpy as np

np.sum(lengths) / 60 / 60

305.8013451851877

In [20]:
import pandas as pd

pd.DataFrame(filtered_filtered).to_parquet('husein-00000-of-00001.parquet')

In [21]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="husein-00000-of-00001.parquet",
    path_in_repo="data/husein-00000-of-00001.parquet",
    repo_id="mesolitica/TTS",
    repo_type="dataset",
)

husein-00000-of-00001.parquet:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/TTS/commit/7340b6b7c7fe01308dd2d87725e4cbc497715452', commit_message='Upload data/husein-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='7340b6b7c7fe01308dd2d87725e4cbc497715452', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/TTS', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/TTS'), pr_revision=None, pr_num=None)

In [1]:
!zip -rq generate-husein-wiki-normalized-v2.zip generate-husein-wiki-normalized-v2
!zip -rq generate-husein-news-normalized-v2.zip generate-husein-news-normalized-v2
!zip -rq husein-news-politics-normalized-v2.zip husein-news-politics-normalized-v2
!zip -rq husein-chatbot-politics-normalized-v2.zip husein-chatbot-politics-normalized-v2
!zip -rq husein-chatbot-normalized-v2.zip husein-chatbot-normalized-v2

In [6]:
from huggingface_hub import HfApi
api = HfApi()

In [7]:
for f in glob('*v2.zip'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id="mesolitica/TTS",
        repo_type="dataset",
    )

husein-news-politics-normalized-v2.zip


husein-news-politics-normalized-v2.zip:   0%|          | 0.00/230M [00:00<?, ?B/s]

husein-chatbot-normalized-v2.zip


husein-chatbot-normalized-v2.zip:   0%|          | 0.00/3.21G [00:00<?, ?B/s]

generate-husein-news-normalized-v2.zip


generate-husein-news-normalized-v2.zip:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

generate-husein-wiki-normalized-v2.zip


generate-husein-wiki-normalized-v2.zip:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

husein-chatbot-politics-normalized-v2.zip


husein-chatbot-politics-normalized-v2.zip:   0%|          | 0.00/2.02G [00:00<?, ?B/s]